In [1]:
""" This notebook 
(1) takes optimal portfolios for ordinary and relative power utilies from the files alpha_ordinary_opt_portf.txt and alpha_relative_opt_portf, computed by the notebooks GDSEG_alpha_ordinary.ipynb and GDSEG_alpha_relative.ipynb  
(2) drops small weights and normalizes the remaining ones
(3) constructs the tables containing optimal weights, total wealth, annual return and annual volatility for each porftolio (Table 4 of the paper)
"""
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Importing NYSE_2 dataset
stocks=pd.read_csv('NYSE_2.csv')
print(stocks.shape)
stocks.head()

(11178, 19)


,ahp,alcoa,amerb,coke,dow,dupont,ford,ge,gm,hp,ibm,inger,jnj,kimbc,merck,mmm,morris,pandg,schlum
0,1.01515,1.02765,1.04183,1.00637,1.00847,1.01983,1.00000,1.00000,1.01026,1.01935,1.00429,1.01357,0.99683,1.05340,1.03148,1.03377,1.01495,1.00775,1.01176
1,1.01493,1.04036,0.98905,1.00475,1.00840,1.00833,1.00157,1.02187,0.99746,1.01266,0.99573,1.00446,1.00318,1.00461,1.00898,1.00251,1.00000,1.00192,1.01938
2,1.00000,0.97629,0.97786,0.98583,0.99722,0.99449,0.98116,0.97860,0.98219,0.98125,0.98571,0.99556,0.95873,0.98165,0.98043,0.95990,0.97218,0.98656,0.97338
3,1.02451,1.00662,1.02642,1.01917,0.99443,1.00693,1.02720,1.00795,0.98705,1.00637,1.01522,1.00000,1.01325,0.98131,1.01089,1.03655,0.99663,1.00778,1.00000
4,1.03100,0.98465,1.00368,1.00313,1.02801,1.00413,1.04361,1.00394,1.00525,1.03165,1.02427,1.01563,1.00654,1.02381,1.01077,0.99496,0.98649,1.01158,1.01563


In [3]:
# r: array for stock returns
N=stocks.shape[0]
d=stocks.shape[1]
r=np.zeros((N,d))
r=stocks.to_numpy()

In [4]:
def refine(w,tol=1/10**3):
    """ Drops small weights and normalizes the remaining ones """
    d=w.shape[0]
    opt_num=[i for i in range(d) if w[i]>tol]
    w=w[opt_num]/np.sum(w[opt_num])    
    return opt_num, w

In [5]:
alphas=[0.01,0.1,0.2,0.3,0.5,0.75]

In [6]:
# Take optimal portfolio weights from the file alpha_ordinary_opt_portf.txt
f=open('alpha_ordinary_opt_portf.txt','r')
n_alphas=sum(1 for line in f)
f.close()
opt_portf=np.zeros((n_alphas,d))
f=open('alpha_ordinary_opt_portf.txt','r')
s=-1
for line in f:
    s+=1
    opt_portf[s,:]=[float(x) for x in line.split()]
f.close()

In [7]:
# wd: dictionary, containing optimal weights after dropping the small weights
# opt_num_d: dictionary, containing the numbers of stocks with large weights
# X[s]: total wealth of the optimal portolio, corrsponding to s-th element of the list alphas=[0.01,0.1,0.2,0.3,0.5,0.75]
wd={}
opt_num_d={}
# X: wealth
X=np.ones(n_alphas)
# Y: annual volatility of log-returns
Y=np.zeros(n_alphas)
z=[]
for s in range(n_alphas):
    opt_num, w = refine(opt_portf[s,:])  
    opt_num_d[s]=opt_num
    wd[s]=w
    for t in range(N):
        X[s]=X[s]*np.dot(w,r[t,opt_num])
        #z.append(np.log(np.dot(w,r[t,opt_num])))
        z.append(np.dot(w,r[t,opt_num]))
    Y[s]=np.std(z)*np.sqrt(252)
    z=[]

In [8]:
# Forming the list of tuples (stock name, optimal portfolio) for each alpha
list_portf=[]
s=-1
tps=[]
for alpha in alphas:
    s+=1
    for z1, z2 in zip(stocks.columns[opt_num_d[s]],wd[s]):
        tps.append((z1,z2))    
    list_portf.append(tps)
    tps=[]
print(list_portf)

[[('hp', 0.17923866465715677), ('morris', 0.7518121129878252), ('schlum', 0.0689492223550179)], [('hp', 0.17617289682388979), ('morris', 0.7765649892426821), ('schlum', 0.04726211393342805)], [('hp', 0.17794447925092566), ('morris', 0.8220555207490744)], [('hp', 0.1589207096440556), ('morris', 0.8410792903559444)], [('hp', 0.09719686836055239), ('morris', 0.9028031316394476)], [('morris', 1.0)]]


In [9]:
# table contains optimal weights, total wealth, annual return and annual volatility for each optimal portolio in the case of the ordinary power utility
# It is the first part of Table 4 of the paper
table_0=pd.DataFrame(index=alphas,data=list_portf,columns=['weight_1','weight_2','weight_3'])
table_1=pd.DataFrame(index=alphas,data=[X[s] for s in range(n_alphas)],columns=['Wealth'])
table_2=pd.DataFrame(index=alphas,data=[X[s]**(252/N) for s in range(n_alphas)],columns=['Annual return'])
table_3=pd.DataFrame(index=alphas,data=Y,columns=['Annual volatility'])
table=pd.concat([table_0,table_1,table_2,table_3],axis=1)
table

,weight_1,weight_2,weight_3,Wealth,Annual return,Annual volatility
0.01,"(hp, 0.17923866465715677)","(morris, 0.7518121129878252)","(schlum, 0.0689492223550179)",4100.404059,1.206281,0.233911
0.10,"(hp, 0.17617289682388979)","(morris, 0.7765649892426821)","(schlum, 0.04726211393342805)",4091.184548,1.206220,0.237427
0.20,"(hp, 0.17794447925092566)","(morris, 0.8220555207490744)",None,4035.731299,1.205849,0.245141
0.30,"(hp, 0.1589207096440556)","(morris, 0.8410792903559444)",None,4016.056743,1.205716,0.246878
0.50,"(hp, 0.09719686836055239)","(morris, 0.9028031316394476)",None,3885.447626,1.204818,0.253959
0.75,"(morris, 1.0)",None,None,3496.681830,1.201958,0.269216


In [10]:
# Take optimal portfolio weights from the file alpha_relative_opt_portf.txt
f=open('alpha_relative_opt_portf.txt','r')
n_alphas=sum(1 for line in f)
f.close()
opt_portf=np.zeros((n_alphas,d))
f=open('alpha_relative_opt_portf.txt','r')
s=-1
for line in f:
    s+=1
    opt_portf[s,:]=[float(x) for x in line.split()]
f.close()

In [11]:
# wd: dictionary, containing optimal weights after after dropping the small weights
# opt_num_d: dictionary, containing the numbers of stocks with large weights
# X[s]: total wealth of the optimal portolio with number s
wd={}
opt_num_d={}
X=np.ones(n_alphas)
z=[]
for s in range(n_alphas):
    opt_num, w = refine(opt_portf[s,:])  
    opt_num_d[s]=opt_num
    wd[s]=w
    for t in range(N):
        X[s]=X[s]*np.dot(w,r[t,opt_num])
        z.append(np.log(np.dot(w,r[t,opt_num])))
    Y[s]=np.std(z)*np.sqrt(252)
    z=[]

In [12]:
# Forming the list of tuples (stock name, optimal portfolio) for each alpha
list_portf=[]
s=-1
tps=[]
for alpha in alphas:
    s+=1
    for z1, z2 in zip(stocks.columns[opt_num_d[s]],wd[s]):
        tps.append((z1,z2))    
    list_portf.append(tps)
    tps=[]
print(list_portf)

[[('hp', 0.1782212816819049), ('morris', 0.7522943759264317), ('schlum', 0.0694843423916635)], [('hp', 0.16169496625739518), ('morris', 0.7881755067050366), ('schlum', 0.050129527037568185)], [('hp', 0.14762887053138254), ('morris', 0.8523711294686174)], [('hp', 0.10690907362543285), ('morris', 0.8930909263745671)], [('morris', 1.0)], [('morris', 1.0)]]


In [13]:
# table contains optimal weights, total wealth, annual return and annual volatility for each optimal portolio in the case of relative power utility
# It is the second part of Table 4 of the paper
table_0=pd.DataFrame(index=alphas,data=list_portf,columns=['weight_1','weight_2','weight_3'])
table_1=pd.DataFrame(index=alphas,data=[X[s] for s in range(n_alphas)],columns=['Wealth'])
table_2=pd.DataFrame(index=alphas,data=[X[s]**(252/N) for s in range(n_alphas)],columns=['Annual return'])
table_3=pd.DataFrame(index=alphas,data=Y,columns=['Annual volatility'])
table=pd.concat([table_0,table_1,table_2,table_3],axis=1)
table

,weight_1,weight_2,weight_3,Wealth,Annual return,Annual volatility
0.01,"(hp, 0.1782212816819049)","(morris, 0.7522943759264317)","(schlum, 0.0694843423916635)",4100.444982,1.206282,0.234094
0.10,"(hp, 0.16169496625739518)","(morris, 0.7881755067050366)","(schlum, 0.050129527037568185)",4085.655992,1.206183,0.238369
0.20,"(hp, 0.14762887053138254)","(morris, 0.8523711294686174)",None,3999.695314,1.205605,0.248230
0.30,"(hp, 0.10690907362543285)","(morris, 0.8930909263745671)",None,3912.532403,1.205007,0.252969
0.50,"(morris, 1.0)",None,None,3496.681830,1.201958,0.269624
0.75,"(morris, 1.0)",None,None,3496.681830,1.201958,0.269624
